# In this notebook we display the performance of damped Newton with preconditioning for different values of the parameters: Number of preconditioning vectors and the number of iterative inversions.

In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)
%matplotlib inline
%load_ext autoreload
%autoreload 

In [ ]:
relative_path_to_new_folder = "../Images"
os.makedirs(relative_path_to_new_folder, exist_ok = True)
if not os.path.isdir('../Images/DampedNewtonPreconditioning_fordiffparam_images'):
    os.makedirs('../Images/DampedNewtonPreconditioning_fordiffparam_images')

In [ ]:
"""To compute distance matrix"""
def distmat( x, y ):
    return np.sum( x**2,0 )[:,None] + np.sum( y**2,0 )[None,:] - 2 * x.transpose().dot( y )

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP( u, K, v ):
    return u[:,None] * K * v[None,:]

def plotp(x, col,plt, scale = 200, edgecolors = "k"):
  return plt.scatter( x[0,:], x[1,:], s = scale, edgecolors = edgecolors,  c = col, cmap = 'plasma', linewidths = 2 )

In [ ]:
def generate_data(N):
    """
     N is a list of the size of the data on x and y
    """
    x = np.random.rand( 2, N[0] ) - 0.5
    theta = 2 * np.pi * np.random.rand( 1, N[1] )
    r = 0.8 + .2 * np.random.rand( 1, N[1] )
    y = np.vstack( ( r * np.cos( theta ), r * np.sin( theta ) ) )
    return x, y


In [ ]:
N = [ 500, 600 ]
x, y = generate_data( N )  

In [ ]:
import computational_OT

# Damped Newton algorithm:
The dual formulation of OT is given by
$$
OT_{\varepsilon} = \max_{f\in \mathbb{R}^{n}, g\in\mathbb{R}^{m}} \langle f, \alpha \rangle + \langle g, \beta \rangle - \varepsilon\left(\langle\alpha \otimes \beta, e^{\frac{f}{\varepsilon}}\odot K \odot e^{\frac{g}{\varepsilon}}  \rangle-1\right)\ ,
$$
where
$$
\alpha \in \mathcal{M}_{1}(\mathcal{X}),\ \beta \in \mathcal{M}_{1}(\mathcal{Y}),\ \varepsilon>0,\ f\in\mathbb{R}^{n},\ g\in \mathbb{R}^{m}\ .
$$

The Hessian is given by 
$\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)=\frac{-1}{\varepsilon}
\begin{pmatrix}
\Delta(\alpha) && \pi_{\varepsilon}\\
\pi^{T}_{\varepsilon} && \Delta(\beta) 
\end{pmatrix}
\ , \ $ where $\pi\mathbb{1}_{m} = \alpha,\ \pi^{T}\mathbb{1}_{n}=\beta,\ $ and $\Delta = diag: \mathbb{R}^{n} \rightarrow M_{n}(\mathbb{R})$ is the linear operator mapping a vector  to a diagonal matrix  containing  this vector.

This implies 
$$
\begin{pmatrix}
\Delta(\alpha) && \pi_{\varepsilon}\\
\pi^{T}_{\varepsilon} && \Delta(\beta) 
\end{pmatrix}
\begin{pmatrix}
\mathbb{1}_{n}\\
\mathbb{1}_{m}
\end{pmatrix} = 0\ ,
$$
that is,
$$
\begin{pmatrix}
\mathbb{1}_{n}\\
\mathbb{1}_{m}
\end{pmatrix}\in \ker(\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g))\ .
$$
Hence, $\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)$ is singular. Therefore, on regularization we have the following Hessian
$
H_{reg} := \nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)+\lambda cc^{T}\ ,
$ 
where $c= \begin{pmatrix}\frac{\mathbb{1}}{\sqrt{n+m}}\\-\frac{\mathbb{1}}{\sqrt{n+m}}\end{pmatrix}\in M_{(n+m),1}(\mathbb{R})$.

Now, at the $k^{th}$ iteration solve
$\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)p_{k} = \nabla Q_{\alpha, \beta,\varepsilon}(f,g)$ to obtain the optimizing direction vector $p_{k}$ and then perform the Armijo condition to obtain the update step $\alpha_{k}$ such that we have the update
$$
(f,g) \leftarrow (f,g) + \alpha_{k} p_{k}\ .
$$


In [ ]:
# Damped Newton
print("Damped Newton... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
dampedNewtonP = []
results_dampedNewton  = []
times_dampedNewton    = []
Hessians_dampedNewton = []
epsilons = [ 1.0, 0.5, 0.1, 0.05, 0.03 ]
#Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton( K,
                                                a,
                                                b,
                                                f,
                                                g,
                                                eps,
                                                rho,
                                                c )
    out = Optimizer._update(    max_iterations = 50,
                                debug = False )
    end = time.time()
    if out != -1:
        results_dampedNewton.append( out )
        times_dampedNewton.append( end - start )
        print( " |- Computing P" )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        dampedNewtonP.append( P_opt )
        print( " |- Recording (unstabilized) Hessian \n" )
        mat  = - eps * Optimizer.Hessian
        diag = 1/np.sqrt( np.concatenate( ( a, b ), axis = None ) )
        mat = diag[:,None] * mat * diag[None,:]
        Hessians_dampedNewton.append( mat )
    else:
        epsilons.remove( eps )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewton)):
  error = np.asarray( results_dampedNewton[i]['error_a'] ) + np.asarray( results_dampedNewton[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/ErrorDampedNewton.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

### Objective function plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "Objective Function" )
for i in range(len(results_dampedNewton)):
  plt.plot( np.asarray( results_dampedNewton[i]["objective_values"] ), label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/ObjectiveDampedNewton.pdf", format = 'pdf' )
plt.show()

### Ascent step-size plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "Alpha" )
for i in range(len(results_dampedNewton)):
  plt.plot( np.asarray( results_dampedNewton[i]['linesearch_steps'] ), label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/AlphaDampedNewton.pdf", format = 'pdf' )
plt.show()

# I. Plotting spectrum as a function of $\varepsilon$

In [ ]:
def spectral_decomposition( mat ):
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort( eig )
    eig = eig[ sorting_indices ]
    v   = v[ : , sorting_indices ]
    print( "List of smallest eigenvalues: ", eig[ : 10 ] )
    print( "List of largest  eigenvalues: ", eig[ - 10 : ] )
    return eig, v

In [ ]:
eigs = []
eigvecs = []
for i in range( len(epsilons) ) :
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    ev = spectral_decomposition( Hessians_dampedNewton[i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")
# end for

In [ ]:
fig, ax = plt.subplots( figsize = ( 5, 12 ), nrows = len(epsilons), ncols = 1, sharey = True )
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
plt.subplots_adjust( wspace = 0, hspace = 0.5 )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/eigenhistunstabilized.pdf", format = 'pdf')
plt.show()

# II. Actual preconditioning

In [ ]:
def build_preconditioners( num_eigs, modified_Hessian, ansatz = True ):
    # Diagonalize
    eigenvalues, eigenvectors = np.linalg.eigh( modified_Hessian )
    sorting_indices = np.argsort( eigenvalues )
    eigenvalues  = eigenvalues[ sorting_indices ]
    eigenvectors = eigenvectors[ : , sorting_indices ]
    # Form null vector
    if not ansatz:
        null_vector = eigenvectors[ : , 0 ]
    else:
        null_vector = np.hstack( ( np.ones(N[0]), - np.ones(N[1]) ) )
        norm = np.sqrt( N[0] + N[1] )
        null_vector = null_vector/norm
    # Form other vectors (only 13)
    _, m = eigenvectors.shape
    indices = []
    for i in range(num_eigs//2):
        indices.append( m - i - 2 )
        indices.append( i + 1 )
    # end for
    if num_eigs%2 != 0:
        indices.append( m - 1 - ( num_eigs//2 ) )
   
    precond_vectors = eigenvectors[ : , indices ]
    precond_vectors = []
    for index in indices:
        precond_vectors.append( eigenvectors[ : ,index ] )
    # end for
    return null_vector, precond_vectors

## Damped Newton with Preconditioning
Here we perform dual damped Newton with preconditioning. Here we consider $t$ eigenvalues of the Hessian that we want to move to one and form the following preconditioning matrix using the corresponding eigenvectors,
$$
P = \left(I_{n+m}-\sum_{i-1}^{t}\left(1 - \frac{1}{\sqrt{\lambda_{i}}}\right)y_{i}y_{i}^{T}\right)\ ,
$$
where
$$
y_{i} \in \ker\left(\nabla^{2}_{f}Q_{\alpha, \beta, \varepsilon}(f)-\lambda_{i}I_{n}\right),\ \forall i= 1,\dots,k\ ,
$$
 are orthonormal.

Now, at the $k^{th}$ iteration we solve the following equation:
$$
(P\nabla^{2}Q_{\alpha, \beta, \varepsilon}(f)P)(Pp_{k})=P\nabla Q_{\alpha, \beta, \varepsilon}(f)\ ,
$$
using iterative inversion methods such as "Conjugate gradient" and "GMRES" to get the ascent direction $p_{k}$, following which we use the Armijo condition to obtain the ascent step size $\alpha_{k}$.

## 5 eigenvalues

In [ ]:
num_eigs = 5
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_dampedNewton[-1], ansatz = False )

### Iterative Inversions:30

In [ ]:
# Damped Newton with preconditioning
print(" Damped Newton with preconditioning... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_I_inv30_5 = []
dampedNewtonwithprecondP_I_inv30_5 = []
results_dampedNewtonwithprecond_I_inv30_5  = []
times_dampedNewtonwithprecond_I_inv30_5    = []
# Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
precond_epsilons = [ 1.0, 0.5, 0.1, 0.05, 0.03 ]
f, g = None, None
for eps in precond_epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )

    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50, 
                                iterative_inversion = 30,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtonwithprecond_I_inv30_5.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_I_inv30_5.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_I_inv30_5.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_I_inv30_5.append( Optimizer.modified_Hessian )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_I_inv30_5) ):
  error = np.asarray( results_dampedNewtonwithprecond_I_inv30_5[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_I_inv30_5[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/ErrorDampedNewton_final_cgIt30_5eig.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### Averaged Time Plot

In [ ]:
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_dampedNewtonwithprecond_I_inv30_5[0]['timings'][0]) ):
  values = []
  for i in range( len(results_dampedNewtonwithprecond_I_inv30_5) ):
    mean = 0
    for k in range( len(results_dampedNewtonwithprecond_I_inv30_5[i]['timings']) ):
      mean += results_dampedNewtonwithprecond_I_inv30_5[i]['timings'][k][j]
    # end for
    mean = mean/len(results_dampedNewtonwithprecond_I_inv30_5[i]['timings']) 
    values.append( mean )
  # end for
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label = text[j],linewidth = 2 )
    plt.legend( loc = "upper right" )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/Timings_DampedNewtonwPrecondition_final_cgIt30_5eig.pdf", format = 'pdf')
plt.show()

### Iterative Inversions:20

In [ ]:
# Damped Newton with preconditioning
print(" Damped Newton with preconditioning... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_I_inv20_5 = []
dampedNewtonwithprecondP_I_inv20_5 = []
results_dampedNewtonwithprecond_I_inv20_5  = []
times_dampedNewtonwithprecond_I_inv20_5    = []
# Cost matrix
C = distmat( x, y ) 
# a and b
a = normalize( np.ones(N[0]) )
b = normalize( np.ones(N[1]) )
precond_epsilons = [ 1.0, 0.5, 0.1, 0.05 ]
f, g = None, None
for eps in precond_epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50, 
                                iterative_inversion = 30,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtonwithprecond_I_inv20_5.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_I_inv20_5.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_I_inv20_5.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_I_inv20_5.append( Optimizer.modified_Hessian )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_I_inv20_5) ):
  error = np.asarray( results_dampedNewtonwithprecond_I_inv20_5[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_I_inv20_5[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/ErrorDampedNewton_final_cgIt20_5eig.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### Averaged Time Plot

In [ ]:
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]

plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_dampedNewtonwithprecond_I_inv20_5[0]['timings'][0]) ):
  values = []
  for i in range( len(results_dampedNewtonwithprecond_I_inv20_5) ):
    mean = 0
    for k in range( len(results_dampedNewtonwithprecond_I_inv20_5[i]['timings']) ):
      mean += results_dampedNewtonwithprecond_I_inv20_5[i]['timings'][k][j]
    # end for
    mean = mean/len(results_dampedNewtonwithprecond_I_inv20_5[i]['timings']) 
    values.append( mean )
  # end for
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label = text[j],linewidth = 2 )
    plt.legend( loc = "upper right" )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/Timings_DampedNewtonwPrecondition_final_cgIt20_5eig.pdf", format = 'pdf' )
plt.show()

### Iterative Inversions:50

In [ ]:
# Damped Newton with preconditioning
print(" Damped Newton with preconditioning... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_I_inv50_5 = []
dampedNewtonwithprecondP_I_inv50_5 = []
results_dampedNewtonwithprecond_I_inv50_5  = []
times_dampedNewtonwithprecond_I_inv50_5    = []
# Cost matrix
C = distmat( x, y ) 
# a and b
a = normalize( np.ones(N[0]) )
b = normalize( np.ones(N[1]) )
precond_epsilons = [ 1.0, 0.5, 0.1, 0.05 ]
f, g = None, None
for eps in precond_epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50, 
                                iterative_inversion = 30,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtonwithprecond_I_inv50_5.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_I_inv50_5.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_I_inv50_5.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_I_inv50_5.append( Optimizer.modified_Hessian )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_I_inv50_5) ):
  error = np.asarray( results_dampedNewtonwithprecond_I_inv50_5[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_I_inv50_5[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/ErrorDampedNewton_final_cgIt50_5eig.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### Averaged Time Plot

In [ ]:
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_dampedNewtonwithprecond_I_inv50_5[0]['timings'][0]) ):
  values = []
  for i in range( len(results_dampedNewtonwithprecond_I_inv50_5) ):
    mean = 0
    for k in range( len(results_dampedNewtonwithprecond_I_inv50_5[i]['timings']) ):
      mean += results_dampedNewtonwithprecond_I_inv50_5[i]['timings'][k][j]
    # end for
    mean = mean/len(results_dampedNewtonwithprecond_I_inv50_5[i]['timings']) 
    values.append( mean )
  # end for
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label = text[j], linewidth = 2 )
    plt.legend( loc = "upper right" )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/Timings_DampedNewtonwPrecondition_final_cgIt50_5eig.pdf", format = 'pdf' )
plt.show()

## 13 eigenvalues

In [ ]:
num_eigs = 13
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_dampedNewton[-1], ansatz=False )

### Iterative Inversions:30

In [ ]:
# Damped Newton with preconditioning
print(" Damped Newton with preconditioning... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_I_inv30_13 = []
dampedNewtonwithprecondP_I_inv30_13 = []
results_dampedNewtonwithprecond_I_inv30_13  = []
times_dampedNewtonwithprecond_I_inv30_13    = []
# Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones(N[0]) )
b = normalize( np.ones(N[1]) )
precond_epsilons = [ 1.0, 0.5, 0.1, 0.05 ]
f, g = None, None
for eps in precond_epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp(-C/eps)
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50, 
                                iterative_inversion = 30,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtonwithprecond_I_inv30_13.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_I_inv30_13.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_I_inv30_13.append( P_opt )  
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_I_inv30_13.append( Optimizer.modified_Hessian )
# end for

#### Error plot

In [ ]:

plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_I_inv30_13) ):
  error = np.asarray( results_dampedNewtonwithprecond_I_inv30_13[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_I_inv30_13[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioningfor $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/ErrorDampedNewton_final_cgIt30_13eig.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### Averaged Time Plot

In [ ]:
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_dampedNewtonwithprecond_I_inv30_13[0]['timings'][0]) ):
  values = []
  for i in range( len(results_dampedNewtonwithprecond_I_inv30_13) ):
    mean = 0
    for k in range( len(results_dampedNewtonwithprecond_I_inv30_13[i]['timings']) ):
      mean += results_dampedNewtonwithprecond_I_inv30_13[i]['timings'][k][j]
    # end for
    mean = mean/len(results_dampedNewtonwithprecond_I_inv30_13[i]['timings']) 
    values.append( mean )
  # end for
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label = text[j],linewidth = 2 )
    plt.legend( loc = "upper right" )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/Timings_DampedNewtonwPrecondition_final_cgIt30_13eig.pdf", format = 'pdf')
plt.show()

### Iterative Inversions:20

In [ ]:
# Damped Newton with preconditioning
print(" Damped Newton with preconditioning... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_I_inv20_13 = []
dampedNewtonwithprercondP_I_inv20_13 = []
results_dampedNewtonwithprecond_I_inv20_13  = []
times_dampedNewtonwithprecond_I_inv20_13    = []
# Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones(N[0]) )
b = normalize( np.ones(N[1]) )
precond_epsilons = [ 1.0, 0.5, 0.1, 0.05 ]
f, g = None, None
for eps in precond_epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50, 
                                iterative_inversion = 20,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtonwithprecond_I_inv20_13.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_I_inv20_13.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprercondP_I_inv20_13.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]    
    final_modified_Hessians_I_inv20_13.append( Optimizer.modified_Hessian )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_I_inv20_13) ):
  error = np.asarray( results_dampedNewtonwithprecond_I_inv20_13[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_I_inv20_13[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/ErrorDampedNewton_final_cgIt20_13eig.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### Averaged Time Plot

In [ ]:
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_dampedNewtonwithprecond_I_inv20_13[0]['timings'][0]) ):
  values = []
  for i in range( len(results_dampedNewtonwithprecond_I_inv20_13) ):
    mean = 0
    for k in range( len(results_dampedNewtonwithprecond_I_inv20_13[i]['timings']) ):
      mean += results_dampedNewtonwithprecond_I_inv20_13[i]['timings'][k][j]
    # end for
    mean = mean/len(results_dampedNewtonwithprecond_I_inv20_13[i]['timings']) 
    values.append( mean )
  # end for
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label=text[j],linewidth = 2 )
    plt.legend( loc = "upper right" )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/Timings_DampedNewtonwPrecondition_final_cgIt20_13eig.pdf", format = 'pdf' )
plt.show()

### Iterative Inversions:50

In [ ]:
# Damped Newton with preconditioning
print(" Damped Newton with preconditioning... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_I_inv50_13 = []
dampedNewtonwithprecondP_I_inv50_13 = []
results_dampedNewtonwithprecond_I_inv50_13  = []
times_dampedNewtonwithprecond_I_inv50_13    = []
# Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones(N[0]) )
b = normalize( np.ones(N[1]) )
precond_epsilons = [ 1.0, 0.5, 0.1, 0.05 ]
f, g = None, None
for eps in precond_epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50, 
                                iterative_inversion = 50,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtonwithprecond_I_inv50_13.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_I_inv50_13.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_I_inv50_13.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_I_inv50_13.append( Optimizer.modified_Hessian )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_I_inv50_13) ):
  error = np.asarray( results_dampedNewtonwithprecond_I_inv50_13[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_I_inv50_13[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/ErrorDampedNewton_final_cgIt50_13eig.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### Averaged Time Plot

In [ ]:
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_dampedNewtonwithprecond_I_inv50_13[0]['timings'][0]) ):
  values = []
  for i in range( len(results_dampedNewtonwithprecond_I_inv50_13) ):
    mean = 0
    for k in range( len(results_dampedNewtonwithprecond_I_inv50_13[i]['timings']) ):
      mean += results_dampedNewtonwithprecond_I_inv50_13[i]['timings'][k][j]
    # end for
    mean = mean/len(results_dampedNewtonwithprecond_I_inv50_13[i]['timings']) 
    values.append( mean )
  # end for
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label = text[j],linewidth = 2 )
    plt.legend( loc = "upper right" )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/Timings_DampedNewtonwPrecondition_final_cgIt50_13eig.pdf", format = 'pdf' )
plt.show()

## 20 eigenvalues

In [ ]:
num_eigs = 20
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_dampedNewton[-1], ansatz = False )

### Iterative Inversions:30

In [ ]:
# Damped Newton with preconditioning
print(" Damped Newton with preconditioning... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_I_inv30_20 = []
dampedNewtonwithprecondP_I_inv30_20 = []
results_dampedNewtonwithprecond_I_inv30_20  = []
times_dampedNewtonwithprecond_I_inv30_20    = []
# Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones(N[0]) )
b = normalize( np.ones(N[1]) )
precond_epsilons = [ 1.0, 0.5, 0.1, 0.05 ]
f, g = None, None
for eps in precond_epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50, 
                                iterative_inversion = 30,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtonwithprecond_I_inv30_20.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_I_inv30_20.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_I_inv30_20.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_I_inv30_20.append( Optimizer.modified_Hessian )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_I_inv30_20) ):
  error = np.asarray( results_dampedNewtonwithprecond_I_inv30_20[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_I_inv30_20[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/ErrorDampedNewton_final_cgIt30_20eig.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### Averaged Time Plot

In [ ]:
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_dampedNewtonwithprecond_I_inv30_20[0]['timings'][0]) ):
  values = []
  for i in range( len(results_dampedNewtonwithprecond_I_inv30_20) ):
    mean = 0
    for k in range( len(results_dampedNewtonwithprecond_I_inv30_20[i]['timings']) ):
      mean += results_dampedNewtonwithprecond_I_inv30_20[i]['timings'][k][j]
      # end for
    mean = mean/len(results_dampedNewtonwithprecond_I_inv30_20[i]['timings']) 
    values.append( mean )
  # end for
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label = text[j], linewidth = 2 )
    plt.legend( loc = "upper right" )
  # end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/Timings_DampedNewtonwPrecondition_final_cgIt30_20eig.pdf", format = 'pdf' )
plt.show()

### Iterative Inversions:20

In [ ]:
# Damped Newton with preconditioning
print(" Damped Newton with preconditioning... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_I_inv20_20 = []
dampedNewtonwithprecondP_I_inv20_20 = []
results_dampedNewtonwithprecond_I_inv20_20  = []
times_dampedNewtonwithprecond_I_inv20_20    = []
# Cost matrix
C = distmat( x, y ) 
# a and b
a = normalize( np.ones(N[0]) )
b = normalize( np.ones(N[1]) )
precond_epsilons = [ 1.0, 0.5, 0.1, 0.05 ]
f, g = None, None
for eps in precond_epsilons:
    # Line Search
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50, 
                                iterative_inversion = 20,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtonwithprecond_I_inv20_20.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_I_inv20_20.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_I_inv20_20.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_I_inv20_20.append( Optimizer.modified_Hessian )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_I_inv20_20) ):
  error = np.asarray( results_dampedNewtonwithprecond_I_inv20_20[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_I_inv20_20[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/ErrorDampedNewton_final_cgIt20_20eig.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### Averaged Time Plot

In [ ]:
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_dampedNewtonwithprecond_I_inv20_20[0]['timings'][0]) ):
  values = []
  for i in range( len(results_dampedNewtonwithprecond_I_inv20_20) ):
    mean = 0
    for k in range( len(results_dampedNewtonwithprecond_I_inv20_20[i]['timings']) ):
      mean += results_dampedNewtonwithprecond_I_inv20_20[i]['timings'][k][j]
    # end for
    mean = mean/len(results_dampedNewtonwithprecond_I_inv20_20[i]['timings']) 
    values.append( mean )
  # end for
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label = text[j],linewidth = 2 )
    plt.legend( loc = "upper right" )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/Timings_DampedNewtonwPrecondition_final_cgIt20_20eig.pdf", format = 'pdf' )
plt.show()

### Iterative Inversions:50

In [ ]:
# Damped Newton with preconditioning
print(" Damped Newton with preconditioning... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_I_inv50_20 = []
dampedNewtonwithprecondP_I_inv50_20 = []
results_dampedNewtonwithprecond_I_inv50_20  = []
times_dampedNewtonwithprecond_I_inv50_20    = []
# Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
precond_epsilons = [ 1.0, 0.5, 0.1, 0.05 ]
f, g = None, None
for eps in precond_epsilons:
    # Line Search
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50, 
                                iterative_inversion = 50,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtonwithprecond_I_inv50_20.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_I_inv50_20.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_I_inv50_20.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_I_inv50_20.append( Optimizer.modified_Hessian )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_I_inv50_20) ):
  error = np.asarray( results_dampedNewtonwithprecond_I_inv50_20[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_I_inv50_20[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/ErrorDampedNewton_final_cgIt50_20eig.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### Averaged Time Plot

In [ ]:
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_dampedNewtonwithprecond_I_inv50_20[0]['timings'][0]) ):
  values = []
  for i in range( len(results_dampedNewtonwithprecond_I_inv50_20) ):
    mean = 0
    for k in range( len(results_dampedNewtonwithprecond_I_inv50_20[i]['timings']) ):
      mean += results_dampedNewtonwithprecond_I_inv50_20[i]['timings'][k][j]
    # end for
    mean = mean/len(results_dampedNewtonwithprecond_I_inv50_20[i]['timings']) 
    values.append( mean )
  # end for
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label = text[j],linewidth = 2 )
    plt.legend( loc = "upper right" )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.savefig( "../Images/DampedNewtonPreconditioning_fordiffparam_images/Timings_DampedNewtonwPrecondition_final_cgIt50_20eig.pdf", format = 'pdf' )
plt.show()